# AKT Net

# AKT Model

# Train Model

In [ ]:
from load_data import DATA, PID_DATA
import numpy as np

n_question = 9 # from prepare_dataset
n_pid = 0 # from prepare_dataset, 0 if not used
seqlen = 200

model_type = 'pid'
batch_size = 64
n_blocks = 1
d_model = 256
dropout = 0.05
kq_same = 1
l2 = 1e-5
maxgradnorm = -1


In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:

def load_train_val_data(dat):
    train_pid = dat.load_data('train_pid.txt')
    # Split train_pid into training and validation data (80/20 split)
    total_samples = train_pid[0].shape[0]
    val_size = int(total_samples * 0.2)  # 20% for validation
    indices = np.arange(total_samples)
    np.random.shuffle(indices)

    train_indices = indices[val_size:]
    val_indices = indices[:val_size]

    # Extract train and validation data
    train_data = [x[train_indices] for x in train_pid]
    val_data = [x[val_indices] for x in train_pid]
    return train_data, val_data

In [ ]:
from EduKTM import AKT
import optuna
from optuna.visualization import plot_param_importances

import matplotlib.pyplot as plt

def objective(trial):
    # Hyperparameters to be tuned
    model_type = trial.suggest_categorical('model_type', ['pid', 'no_pid'])
    n_blocks = trial.suggest_int('n_blocks', 1, 3)
    d_model = trial.suggest_categorical('d_model', [64, 128, 256, 512])
    dropout = trial.suggest_float('dropout', 0.01, 0.2)
    kq_same = trial.suggest_categorical('kq_same', [0, 1])
    l2 = trial.suggest_float('l2', 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    maxgradnorm = trial.suggest_categorical('maxgradnorm', [-1, 0.1, 0.5, 1.0, 5.0])
    
    if model_type == 'pid':
        dat = PID_DATA(n_question=n_question, seqlen=seqlen, separate_char=',')
    else:
        dat = DATA(n_question=n_question, seqlen=seqlen, separate_char=',')
        
    
    train_data, val_data = load_train_val_data(dat)
    
    # Create and train model with the suggested hyperparameters
    akt = AKT(n_question, n_pid, n_blocks, d_model, dropout, kq_same, l2, batch_size, maxgradnorm)
    akt.train(train_data, val_data, epoch=10, lr=lr)  # Using just 1 epoch for faster tuning
    
    # Evaluate and return AUC as the optimization metric
    _, auc, _ = akt.eval(val_data)
    return auc

# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # Adjust n_trials as needed

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Visualize parameter importances
plt.figure(figsize=(10, 6))
plot_param_importances(study)
plt.title("Parameter Importances")
plt.tight_layout()
plt.show()

In [ ]:
if(study.best_params['model_type'] == 'pid'):
    dat = PID_DATA(n_question=n_question, seqlen=seqlen, separate_char=',')
else:
    dat = DATA(n_question=n_question, seqlen=seqlen, separate_char=',')


train_data, val_data = load_train_val_data(dat)
test_data = dat.load_data('test_pid.txt')


akt = AKT(n_question, n_pid, n_blocks, d_model, dropout, kq_same, l2, batch_size, maxgradnorm)
akt.train(train_data, val_data, epoch=2)
akt.save("akt.params")

In [ ]:
akt.load("akt.params")
_, auc, accuracy = akt.eval(test_data)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))